# GBH Earmarks Python Workbook

This is a workbook which has all data calculations & transforming for earmarks.

In [3]:
# Imports

import pandas as pd
# import geopandas as gpd
import matplotlib.pyplot as plt
import openpyxl # for spreadsheets
import matplotlib.font_manager as fm
import numpy as np
import difflib
import re

In [3]:
# Options to support viewing of more items in console

pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:20,.5f}'.format

In [4]:
# Load data

earmarks = pd.read_csv("data/earmarks.csv")
# amendment_proposers = pd.read_excel("data/amendments.xlsx") <- used later
earmarks.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/amendments.xlsx'

## Population fixing

The population information we get from the US Census isn't perfect -- this particular table, which details all demographic data, including Hispanic or Latino and all other races ignoring those is quite dense and has a bunch of N/A rows since they're separated into different towns. Fortunately, a relatively simple regex call will allow us to get rid of much of this and only filter out rows that we want and the totals for each.

The first cell is mainly for cleaning, the second for calculating diversity, which is going to be explained below.


In [22]:
# Clean population sheet

population = pd.read_csv("data/ma_census/population.csv")
population = population.dropna()

population["Label (Grouping)"] = population["Label (Grouping)"].str.slice(start=4)
population = population.rename(columns={"Label (Grouping)": "city"}, errors="raise")
population = population.rename(mapper=lambda x: x.strip().replace('!', ''), axis='columns')
new_names = {}
for c in population["city"]:
    if type(c) == int or c == "County subdivisions not defined":
        continue
    if c == "Massachusetts":
        new_names[c] = "Massachusetts"
    else:
        new_names[c] = re.sub(r" (Town)? ?(town)? ?(city)?, \w* County, Massachusetts$", "", c) # Replaces "Massachusetts"
population = population.replace(new_names)
population = population.loc[~population["city"].str.contains("County subdivisions not defined,")]

# Annoyingly, all the numbers are comma separated


for col in population.columns[1:]:
    population[col] = population[col].replace(",", "", regex=True).astype(int)
# population.sort_values(by="city")
population.to_csv(path_or_buf="out/cities.csv")

# Demographics Indepth Analysis

The below cells calculate how diverse a given town is. How that's calculated is using the US Census' way of calculating diversity. Note that people with more than 1 listed race are not included in the calculation.

The following are summed to calculate out of the total (Non Hispanic/Latino if unnoted):
- Hispanic/Latino (col 2)
- White (col 5)
- Black/African American (col 6)
- American Indian/Alaska Native (col 7)
- Asian (col 8)
- Native Hawaiian/Pacific Islander (col 9)
- Some Other Race (col 10)
- Two or more Races (col 11)

The census' way of determining diversity is by the Diversity Index, a measure between 0 and 1 on how likely two random people from the population will be of the same race. That in particular is very hard to determine, especially if the two people selected are determined to have two or more races. For more information on how this is determined, see the link to [their blog here](https://www.census.gov/newsroom/blogs/random-samplings/2021/08/measuring-racial-ethnic-diversity-2020-census.html).

Two or more races is determined as one group in the census calculations, and that generally makes sense since if a person has more than one race, say White and African American and are matched with African American, they are technically matched with someone from a different group. Via Bayes rule,

$$Pr(A \cap B) = Pr(A) Pr(B|A)$$ given A and B are in the same group. We can just subtract this probability across the sum of all groups, since that's the probability that a given area has two people picked differently. We can calculate this easily.

The TL;DR of this is we're calculating our own Diversity Index for each town based on the population counts, using probabilities.

In [9]:
columns_to_pick = [2, 5, 6, 7, 8, 9, 10, 11]

def diversity_calculation(row):
    # print(row)
    """Calculates the diversity index for a given city, given by the row above."""
    bins = [row[k] for k in columns_to_pick]
    di_score = 0.
    total = sum(bins)
    for i in range(len(bins)):
        di_score += (bins[i] / total) * ((bins[i]-1) / total)
    return 1 - di_score

# testing by statewide first -- expected value according to the census is 51.6%
cols = population.columns
statewide = pd.DataFrame(index=range(1), columns=range(len(cols)))
for i in columns_to_pick:
    statewide.iloc[0, i] = sum(population[cols[i]])
diversity_calculation(statewide)

0    0.515979
dtype: object

In [12]:
# Now to do this for every town:
population["diversity_score"] = population.apply(lambda row: diversity_calculation(row), raw=True, axis=1)
population.sort_values(by="diversity_score", inplace=True, ascending=False)
population.to_csv(path_or_buf="out/cities_with_diversity.csv") # Unfortunately, the diversity score is at the end, reorder it so it's visible

In [13]:
# Basic statistics

median = earmarks["Amount"].median()
mean = earmarks["Amount"].mean()
stddev = earmarks["Amount"].std()
print(mean, stddev, median) # stddev)

402090.382387022 2103267.934430257 100000.0


In [15]:
# Agency cleanup -- removes the agency description and compresses down to the standard 3 word letter.

def agency_code(row):
    return row["Administering State Agency"][:3]

earmarks["Agency"] = earmarks.apply (lambda row: agency_code(row), axis=1)
earmarks["Administering State Agency"].value_counts()
earmarks
# TODO: group by agency

,Unnamed: 0,Earmark Description,Location,Amount,Administering State Agency,State Agency Contact,Account Number,Earmark Language,Agency,Categories,...,Occupational,Recreation,Rehabilitation,Religious,Transport,Technology,Youth Programs,Accessibility,Maintenance,Research
0,0,54th Massachusetts Reenactors and Historical S...,Boston,25000,MMP - Massachusetts Marketing Partnership,Phyllis.cahaly@mass.gov,70021530,"provided further that not less than $25,000 sh...",MMP,Historical,...,False,False,False,False,False,False,False,False,False,False
1,1,A Street Pier Boat Ramp Rebuilding,Hull,150000,DCR - Department Of Conservation And Recreation,max.j.tassinari@mass.gov,15992031,"provided further, that not less than $150,000 ...",DCR,"Recreation, Construction, Maintenance",...,False,True,False,False,False,False,False,False,True,False
2,2,Abigail Adams Historical Society,Weymouth,25000,ANF - Executive Office For Administration And ...,daniel.shark@mass.gov,15992058,"provided further, that not less than $25,000 s...",ANF,Historical,...,False,False,False,False,False,False,False,False,False,False
3,3,Abington Housing Authority Upgrades,Abington,85000,DHCD - Dept Of Housing And Community Development,alain.fabo@mass.gov,70049318,"provided further, that not less than $85,000 s...",DHC,"Housing, Maintenance",...,False,False,False,False,False,False,False,False,True,False
4,4,Abington Utility Task Vehicle,Abington,28000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992043,"provided further, that not less than $28,000 s...",HED,Transport,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,858,Wrentham Public Water Supply,Wrentham,150000,DEP - Department Of Environmental Protection,steven.mccurdy@mass.gov,20000015,"provided further, that not less than $150,000 ...",DEP,"Environment, Research",...,False,False,False,False,False,False,False,False,False,True
859,859,Youth Community Center in Georgetown,Georgetown,25000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,70100013,"provided further, that not less than $25,000 s...",HED,"Youth Programs, Human Services",...,False,False,False,False,False,False,True,False,False,False
860,860,Youth Development in Lawrence,Lawrence,200000,LWD - Executive Office Of Labor And Workforce ...,sheila.l.tunney2@mass.gov,70100013,"provided further, that not less than $200,000 ...",LWD,Youth Programs,...,False,False,False,False,False,False,True,False,False,False
861,861,YWCA Funding,Cambridge,4500000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992047,"and provided further, that not less than $4,50...",HED,"Economic, Maintenance, Human Services, Equalit...",...,False,False,False,False,False,False,False,False,True,False


# Keyword & Category Matching
The following cells below are all keywords to identify category matching. While there's a good list of keywords that will easily match below, I just wanted to give a quick rationale/things that you might find in these categories.

### Categories
#### Essentials
- Health: Anything relating to COVID-19, healthcare, or protection related to COVID-19, like HVAC installation and testing.
- Housing: All projects related to affordable housing, or other initiatives related to housing
- Food: All projects related to food pantries or access to food

#### Non-Essentials
While the overarching category is non-essentials, they are more broader categories that should not necessarily have direct impacts from the pandemic; mostly these are initiatives for public services unrelated to the above
- Arts: Mostly theater and stage productions, or other cultural initiatives. Includes Museums, Theaters, etc.
- Construction: Any project related to construction
- Economic: Business, commerce, or other economic incentives
- Education: Anything related to Primary/Secondary education, or libraries
- Environment: Many projects in these earmarks correlate to rehabilitation of beaches, or improvement of water or sewage networks.
- Essential Services: Ironically, these are closer to improvements towards fire/police stations, not necessarily funding for them
- Equality: Most projects that are geared towards minority groups or societies. Also includes advocacy groups and initiatives designed to bring up veterans.
- Events: Things related to one-time events like parades, new years parties
- Historical: You'd be surprised how many there are with just one keyword
- Human Services: Any project relating to family, youth, teen, or public safety development.
- Mental Health: Anything related to behavioral health
- Occupational: Projects relating to job training or initiatives to help people find work
- Recreation: Projects relating to recreational facilities such as parks, sports centers, and even sidewalks
- Rehabilitation: Anything related to drug rehabilitation
- Religious: Anything related to church or religious
- Transport: All projects relating to transport. Keywords omit "rail" as many could be projects for rail trails.
- Technology: Many projects in earmarks are related to upgrading facilities with new technology

#### Accessibility, Maintenance and Research
I've specifically separated these three as they're more broad reaching categories that could fit in any of the above. But these are really just improvements or studies.
- Accessibility: Any project relating to improving the quality of life for handicap people
- Maintenance: Any project relating to literally repairing, replacing, upgrading, restoring, improving, rebuilding, etc.
- Research: Any project relating to a feasibility study or a normal study

These categories are generally in flux as I add keywords to better represent all projects, and I will sometimes update manually to fix exceptions.

In [16]:
# Keyword Matching

# NOTE: the current, most updated earmarks_categories have also been manually scanned, and there are roughly 100 items not categorized through the systematic process or mistagged. While I intend to add more categories sometimes, bugs of old categorizing may still remain.

keywords_to_categories = {

    # essentials

    "Health": ["covid", "testing", "primary care", "HVAC", "health", "emergency", "paramedic"],
    "Housing": ["housing", "home", "homeless", "relocation"],
    "Food": ["food", "pantry", "servings"],

    # non-essentials

    "Arts": ["arts", "theater", "stage", "cultur", "museum", "exhibit", "media"],
    "Construction": ["building", "construction"],
    "Economic": ["commerce", "economic", "business"],
    "Education": ["school", "library", "books", "tutoring", "education", "academic"],
    "Environment": ["water", "beach", "sewer", "environment", "flood", "storm", "invasive species", "conservation", "wildlife", "climate", "solar"],
    "Essential Services": ["police department", "fire department"],
    "Equality": ["black", "asian", "women", "haiti", "hispanic", "latin", "LGBT", "seniors", "elders", "diversity", "veterans", "equity", "equality", "naacp", "caribbean", "legion", "disproportionate", "vfw"],
    "Events": ["first night", "celebration", "parade"],
    "Historical": ["historic", "history", " war "],
    "Human Services": ["family", "elder", "senior", "aging", "public safety", "immigrant", "community center", "age", "veterinar", "translation", "domestic violence"],
    "Mental Health": ["mental health" "behavioral health"],
    "Occupational": ["training", "job", "work"],
    "Recreation": ["park", "trail", "sidewalk", "garden", "tennis", "basketball", "soccer", "sports", "pond", "tree", "lighting", "walk", "arboretum"],
    "Rehabilitation": ["addiction", "substance"],
    "Religious": ["temple", "church", "ministry", "jewish",],
    "Transport": ["transport", "vehicle", "road", "train", "bus", "route"],
    "Technology": ["tech", " it ", "tech", "paperless", "fiber optic", "cable", "broadband", "digital", "website", "wireless", "electric", "software"],
    "Youth Programs": ["youth", "ymca", "boys and girls", "teen", "children"],

    # upkeep, research, acessibility
    "Accessibility": ["blind", "handicap", "ADA", "accessible"],
    "Maintenance": ["maintenance", "repair", "replacement", "upgrade", "upkeep", "restoration", "improvement", "rebuild", "renovation", "preservation", "clean"],
    "Research": ["research", "survey", "study", "assessment", "project", "design", "plan"]
}

keywords = categories = keywords_to_categories.keys()


In [17]:
# Matches keywords to earmarks function

def match_keywords(row):
    def in_category(keyword_list, text):
        for k in keyword_list:
            if k in text.lower():
                return True
        return False

    desc = row["Earmark Description"]
    lang = row["Earmark Language"]
    desc_matches = set([k for k in keywords_to_categories if in_category(keywords_to_categories[k], desc)])
    lang_matches = set([k for k in keywords_to_categories if in_category(keywords_to_categories[k], lang)])

    return desc_matches.union(lang_matches)

In [18]:
# Matching keywords to categories, new column for categories

f = lambda x: ', '.join(str(k) for k in match_keywords(x))
cats = earmarks.apply(f, axis=1)
earmarks["Categories"] = cats
earmarks

,Unnamed: 0,Earmark Description,Location,Amount,Administering State Agency,State Agency Contact,Account Number,Earmark Language,Agency,Categories,...,Occupational,Recreation,Rehabilitation,Religious,Transport,Technology,Youth Programs,Accessibility,Maintenance,Research
0,0,54th Massachusetts Reenactors and Historical S...,Boston,25000,MMP - Massachusetts Marketing Partnership,Phyllis.cahaly@mass.gov,70021530,"provided further that not less than $25,000 sh...",MMP,Historical,...,False,False,False,False,False,False,False,False,False,False
1,1,A Street Pier Boat Ramp Rebuilding,Hull,150000,DCR - Department Of Conservation And Recreation,max.j.tassinari@mass.gov,15992031,"provided further, that not less than $150,000 ...",DCR,"Construction, Recreation, Maintenance",...,False,True,False,False,False,False,False,False,True,False
2,2,Abigail Adams Historical Society,Weymouth,25000,ANF - Executive Office For Administration And ...,daniel.shark@mass.gov,15992058,"provided further, that not less than $25,000 s...",ANF,Historical,...,False,False,False,False,False,False,False,False,False,False
3,3,Abington Housing Authority Upgrades,Abington,85000,DHCD - Dept Of Housing And Community Development,alain.fabo@mass.gov,70049318,"provided further, that not less than $85,000 s...",DHC,"Housing, Maintenance",...,False,False,False,False,False,False,False,False,True,False
4,4,Abington Utility Task Vehicle,Abington,28000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992043,"provided further, that not less than $28,000 s...",HED,Transport,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,858,Wrentham Public Water Supply,Wrentham,150000,DEP - Department Of Environmental Protection,steven.mccurdy@mass.gov,20000015,"provided further, that not less than $150,000 ...",DEP,"Environment, Research",...,False,False,False,False,False,False,False,False,False,True
859,859,Youth Community Center in Georgetown,Georgetown,25000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,70100013,"provided further, that not less than $25,000 s...",HED,"Youth Programs, Human Services",...,False,False,False,False,False,False,True,False,False,False
860,860,Youth Development in Lawrence,Lawrence,200000,LWD - Executive Office Of Labor And Workforce ...,sheila.l.tunney2@mass.gov,70100013,"provided further, that not less than $200,000 ...",LWD,Youth Programs,...,False,False,False,False,False,False,True,False,False,False
861,861,YWCA Funding,Cambridge,4500000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992047,"and provided further, that not less than $4,50...",HED,"Maintenance, Equality, Arts, Human Services, E...",...,False,False,False,False,False,False,False,False,True,False


In [19]:
# Quick outputs and fix locations

# for cat in categories:
#     earmarks[cat] = earmarks["Categories"].str.contains(cat)

# Fix towns
town_fixes = {"Greater Boston": "Boston",
         "Padanram": "Dartmouth",
         "MetroWest": "Framingham",
         "Natick & Framingham": "Framingham",
         "Western Mass": "Hatfield",
         "Hilltown": "Huntington",
         "Malden & Revere": "Malden",
         "Burnham": "Methuen",
         "Amesbury, Lawrence": "Newburyport",
         "Nashoba Valley": "Westford"}

town_new_names = {new_names[k]: new_names[k] for k in new_names}

town_fixes.update(town_new_names) # adds all old names from before

print(town_fixes)

# Rematch towns
earmarks["Location"] = earmarks["Location"].map(town_fixes, na_action="ignore")

# DO NOT UNCOMMENT THE FOLLOWING LINES UNLESS YOU INTEND TO REFRESH ALL ITEMS' CATEGORIES!

# earmarks.to_csv(path_or_buf="out/earmarks_categories.csv")
# earmarks.loc[pd.isna(earmarks["Location"])].to_csv(path_or_buf="out/earmarks_without_towns.csv")

{'Greater Boston': 'Boston', 'Padanram': 'Dartmouth', 'MetroWest': 'Framingham', 'Natick & Framingham': 'Framingham', 'Western Mass': 'Hatfield', 'Hilltown': 'Huntington', 'Malden & Revere': 'Malden', 'Burnham': 'Methuen', 'Amesbury, Lawrence': 'Newburyport', 'Nashoba Valley': 'Westford', 'County subdivisions not defined, Barnstable County, Massachusetts': 'County subdivisions not defined, Barnstable County, Massachusetts', 'Barnstable': 'Barnstable', 'Bourne': 'Bourne', 'Brewster': 'Brewster', 'Chatham': 'Chatham', 'Dennis': 'Dennis', 'Eastham': 'Eastham', 'Falmouth': 'Falmouth', 'Harwich': 'Harwich', 'Mashpee': 'Mashpee', 'Orleans': 'Orleans', 'Provincetown': 'Provincetown', 'Sandwich': 'Sandwich', 'Truro': 'Truro', 'Wellfleet': 'Wellfleet', 'Yarmouth': 'Yarmouth', 'Adams': 'Adams', 'Alford': 'Alford', 'Becket': 'Becket', 'Cheshire': 'Cheshire', 'Clarksburg': 'Clarksburg', 'Dalton': 'Dalton', 'Egremont': 'Egremont', 'Florida': 'Florida', 'Great Barrington': 'Great Barrington', 'Hanco

# Amendment additions

Amendments were proposed to this bill and we need to actually account for these -- the sheet at amendments.xlsx has four categories and various people proposing each item

In [20]:
# This refreshes the earmarks & populations dataframes!!!

house_amends = pd.read_excel("data/amendments_house.xlsx")
senate_amends = pd.read_excel("data/amendments_senate.xlsx")
senate_amends = senate_amends.rename(columns={"Category": "Subject", "Bundle": "Consolidated Amendment"})

amendments = pd.concat([house_amends, senate_amends])

earmarks = pd.read_csv("out/earmarks_categories.csv")
population = pd.read_csv("data/cities.csv")
# amendments

amendments_consol = amendments.loc[amendments["Status"] == "Consolidated"]
amendments_adopt =  amendments.loc[amendments["Status"] == "Adopted"]
amendments = pd.concat([amendments_adopt, amendments_consol])

earmarks_temp_merge = earmarks

# Match titles?
# earmarks_temp_merge["Earmarks Desc Lower"] = earmarks_temp_merge["Earmark Description"].str.lower()
# amendments["Earmarks Desc Lower"] = amendments["Title"].str.lower()
#
# amendments = amendments.rename(columns={"Title":"Earmarks Desc Lower"}, errors="raise")

def get_best_match(x):
    matches = difflib.get_close_matches(x, amendments['Title'], cutoff=0.5)
    if len(matches) == 0:
        return pd.NA
    return matches[0]

earmarks_temp_merge['amdt_match'] = earmarks_temp_merge['Earmark Description']
earmarks_temp_merge['amdt_match'].apply(lambda x: get_best_match(x))

# print(amendments.columns, "\n",  earmarks_temp_merge.columns)

earmarks_amdts = earmarks_temp_merge.merge(amendments, left_on="amdt_match", right_on="Title")
# earmarks_amdts.drop("Earmark Desc Lower")

earmarks_amdts.to_csv(path_or_buf="out/amendments_to_legis.csv")
earmarks_amdts.to_excel("out/amendments_to_legis.xlsx")
print(len(earmarks_amdts))
earmarks.drop("amdt_match")
# earmarks_amdts

307


KeyError: "['amdt_match'] not found in axis"

In [ ]:
# Investigate a certain senator?

legis = "Oliveira, Jacob R. (HOU)"
earmarks_amdts[earmarks_amdts["Sponsor"] == legis]
# amendments[amendments["Sponsor"] == legis]

# Category Statistics

Below, individual category statistics are calculated, like the percentage of money contributed to a certain category, the amount, etc.
Note that these altogether will not add up to 100%, since many categories overlap between projects.

In [ ]:
# Categories

# Slow matching process, but it's necessary
def sum_category(category, df):
    total = 0
    for i, row in df.iterrows():
        if category in row["Categories"]:
            total += row["Amount"]
    return total

def count_category(category, df):
    count = 0
    for i, row in df.iterrows():
        if category in row["Categories"]:
            count += 1
    return count

def max_category(category, df, index_filter=None):
    if index_filter is None:
        index_filter = []
    total = 0
    earmark = ""
    location = ""
    index = -1
    for i, row in df.iterrows():
        if i in index_filter:
            continue
        if category in row["Categories"] and row["Amount"] > total:
            total = row["Amount"]
            earmark = row["Earmark Description"]
            location = row["Location"]
            index = i
    return total, earmark, index

total_dist = earmarks["Amount"].sum()
category_stats = pd.DataFrame({"keyword":[], "count":[], "pct_of_earmarks":[], "amount":[], "pct_of_total":[], "average":[], "max":[], "max_project": [], "max_project_index":[]})

for c in categories:
    sum_cat = sum_category(c, earmarks)
    count = count_category(c, earmarks)
    average = sum_cat / count
    # You can remove the index filter, but this just specifically removes projects that clog the "Max Project" stuff like the MBTA and other things in this example, you can totally add more indices if they give you headaches in the data
    max_num, max_earmark, max_index = max_category(c, earmarks)
    category_stats.loc[len(category_stats)] = [c, count, count/len(earmarks), sum_cat, sum_cat/total_dist, average, max_num, max_earmark, max_index]

category_stats.sort_values(by=['amount'], ascending=False)

In [37]:
# Split categories into individual fields

for cat in categories:
    earmarks[cat] = earmarks["Categories"].str.contains(cat)

earmarks.to_csv(path_or_buf="out/earmarks_categories.csv")
earmarks.to_excel("out/earmarks_categories.xlsx")
earmarks

,Unnamed: 0,Earmark Description,Location,Amount,Administering State Agency,State Agency Contact,Account Number,Earmark Language,Agency,Categories,...,Recreation,Rehabilitation,Religious,Transport,Technology,Youth Programs,Accessibility,Maintenance,Research,amdt_match
0,0,54th Massachusetts Reenactors and Historical S...,Boston,25000,MMP - Massachusetts Marketing Partnership,Phyllis.cahaly@mass.gov,70021530,"provided further that not less than $25,000 sh...",MMP,Historical,...,False,False,False,False,False,False,False,False,False,54th Massachusetts Reenactors and Historical S...
1,1,A Street Pier Boat Ramp Rebuilding,Hull,150000,DCR - Department Of Conservation And Recreation,max.j.tassinari@mass.gov,15992031,"provided further, that not less than $150,000 ...",DCR,"Recreation, Construction, Maintenance",...,True,False,False,False,False,False,False,True,False,A Street Pier Boat Ramp Rebuilding
2,2,Abigail Adams Historical Society,Weymouth,25000,ANF - Executive Office For Administration And ...,daniel.shark@mass.gov,15992058,"provided further, that not less than $25,000 s...",ANF,Historical,...,False,False,False,False,False,False,False,False,False,Abigail Adams Historical Society
3,3,Abington Housing Authority Upgrades,Abington,85000,DHCD - Dept Of Housing And Community Development,alain.fabo@mass.gov,70049318,"provided further, that not less than $85,000 s...",DHC,"Housing, Maintenance",...,False,False,False,False,False,False,False,True,False,Abington Housing Authority Upgrades
4,4,Abington Utility Task Vehicle,Abington,28000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992043,"provided further, that not less than $28,000 s...",HED,Transport,...,False,False,False,True,False,False,False,False,False,Abington Utility Task Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,858,Wrentham Public Water Supply,Wrentham,150000,DEP - Department Of Environmental Protection,steven.mccurdy@mass.gov,20000015,"provided further, that not less than $150,000 ...",DEP,"Environment, Research",...,False,False,False,False,False,False,False,False,True,Wrentham Public Water Supply
859,859,Youth Community Center in Georgetown,Georgetown,25000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,70100013,"provided further, that not less than $25,000 s...",HED,"Youth Programs, Human Services",...,False,False,False,False,False,True,False,False,False,Youth Community Center in Georgetown
860,860,Youth Development in Lawrence,Lawrence,200000,LWD - Executive Office Of Labor And Workforce ...,sheila.l.tunney2@mass.gov,70100013,"provided further, that not less than $200,000 ...",LWD,Youth Programs,...,False,False,False,False,False,True,False,False,False,Youth Development in Lawrence
861,861,YWCA Funding,Cambridge,4500000,HED - Executive Office Of Economic Development,rory.c.ohanlon@mass.gov,15992047,"and provided further, that not less than $4,50...",HED,"Economic, Maintenance, Human Services, Equalit...",...,False,False,False,False,False,False,False,True,False,YWCA Funding


In [ ]:
# Housing - Gross Rent according to the American Community Survey

rent_by_income = pd.read_csv("data/ma_census/acs_rent/rent.csv", header=1)

rent_by_income_headers = pd.read_csv("data/ma_census/acs_rent/headers.csv")
# new_headers = dict(zip(rent_by_income_headers["GEO_ID"], rent_by_income_headers["id"]))

rent_by_income["Geographic Area Name"] = rent_by_income["Geographic Area Name"].replace(new_names)
rent_by_income.to_csv(path_or_buf="out/rent_fixed_towns.csv")
rent_by_income.to_excel("out/rent_fixed_towns.xlsx")

mortgage = pd.read_csv("data/ma_census/acs_mortgage/mortgages.csv", header=1)
mortgage["Geographic Area Name"] = rent_by_income["Geographic Area Name"].replace(new_names)
cols = list(mortgage) # Fix row order
cols = [cols[-1]] + cols[:-1]
mortgage = mortgage[cols]
mortgage = mortgage.loc[1:]
mortgage.to_csv(path_or_buf="out/mortgage_fixed_towns.csv")
mortgage.to_excel("out/mortgage_fixed_towns.xlsx")

# Fix towns
rent_by_income = rent_by_income.replace(new_names)
income_on_housing = pd.DataFrame()

# [ for k in range(rent_by_income)]

In [ ]:
earmarks = pd.read_csv("out/earmarks_categories.csv")
earmarks.loc[earmarks["Arts"] == True]
# amendments.loc[amendments["Sponsor" == "Barrett, "]]

In [80]:
arts_projects = earmarks.loc[earmarks["Arts"] == True]
money_per_town = pd.DataFrame()

money_per_town["City"] = population["city"]

# print(money_per_town.columns)
for cat in categories:
    df_cat = earmarks.loc[earmarks[cat] == True]
    money_per_town = pd.merge(money_per_town, df_cat.groupby(['Location']).sum()[["Amount", cat]], left_on="City", right_on="Location", how="left")
    transform_cols = {"Amount": cat + " Amount", cat: cat + " Projects"}

    old_cols = list(transform_cols.keys())
    money_per_town = money_per_town.fillna(0)
    # print(money_per_town)
    for c in old_cols:
        money_per_town[c] = money_per_town.loc[:, c].astype(int)

    money_per_town = money_per_town.rename(columns=transform_cols)

money_per_town.to_csv(path_or_buf="out/money_per_project.csv")
money_per_town.to_excel("out/money_per_project.xlsx")
money_per_town

,City,Health Amount,Health Projects,Housing Amount,Housing Projects,Food Amount,Food Projects,Arts Amount,Arts Projects,Construction Amount,...,Technology Amount,Technology Projects,Youth Programs Amount,Youth Programs Projects,Accessibility Amount,Accessibility Projects,Maintenance Amount,Maintenance Projects,Research Amount,Research Projects
0,Barnstable,160000,2,2500000,5,0,0,400000,1,400000,...,1060000,4,120000,2,0,0,220000,3,850000,3
1,Bourne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,240000,3,0,0
2,Brewster,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Chatham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dennis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,West Boylston,0,0,0,0,0,0,0,0,130000,...,0,0,0,0,0,0,0,0,130000,1
347,West Brookfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
348,Westminster,100000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,50000,1,0,0
349,Winchendon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,100000,1,0,0


In [ ]:
mpc = earmarks["Location"].value_counts().rename_axis("Location").reset_index(name="counts")
mpc = pd.merge(mpc, earmarks.groupby("Location").sum()["Amount"], on="Location").sort_values(by="Amount", ascending=False)